<a href="https://colab.research.google.com/github/swapyswap/Tumor_Detection-/blob/main/Cancer_Detection_using_Neural_Networks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget https://www.dropbox.com/s/78tdjt4a9xdwmcx/archive%20%282%29.zip?dl=0

In [ ]:
!unzip arch*

In [3]:
import pandas as pd
import numpy as np
import shutil
import glob
import os
import math
import warnings
warnings.filterwarnings(action="ignore")
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from sklearn.preprocessing import StandardScaler 
from sklearn.model_selection import train_test_split 
from sklearn.metrics import classification_report
#tensorflow libraries
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import MaxPooling2D, Dense, Dropout,Flatten, Conv2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import TensorBoard,EarlyStopping 

In [4]:
#count the number of images in the respective classes
ROOT_DIR="/content/Brain Tumor Data Set/Brain Tumor Data Set"
number_of_images={}
for dir in os.listdir(ROOT_DIR):
  number_of_images[dir]=len(os.listdir(os.path.join(ROOT_DIR,dir)) )


In [ ]:
number_of_images.items()

We splitted the data in such a way that:-
70% for Train(training) data
15% for validation
15% for testing

In [6]:
def datafolder(p,split):                                                                                                  #Niranjan
   #we will create a folder for storing data for train test
  if not os.path.exists("./"+p):
   os.mkdir("./"+p)

   for dir in os.listdir(ROOT_DIR):
    os.makedirs("./"+p+"/"+dir)
    for img in np.random.choice(a=os.listdir(os.path.join(ROOT_DIR, dir)),size=(math.floor(split*number_of_images[dir])-5),replace=False):
      O=os.path.join(ROOT_DIR,dir,img)
      D=os.path.join("./"+p,dir)
      shutil.copy(O,D)
      os.remove(O)
  else: 
     print(f"{p}folder exists")
      

In [7]:
datafolder("train",0.7)

In [8]:
datafolder("val",0.15)

In [9]:
datafolder("test",0.15)

# MODEL BUILD

In [10]:
from keras.layers import Conv2D,MaxPool2D,Dropout,Flatten,Dense,BatchNormalization,GlobalAvgPool2D
from keras.models import Sequential 
from keras.preprocessing.image import ImageDataGenerator
import keras 

In [11]:
#model.compile(optimizer='adam', loss= keras.losses.binary_crossentropy,metrics=['accuracy'])

## PREPARING OUR DATA USING DATA GENERATOR

In [12]:
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.mobilenet import preprocess_input

In [13]:
def preprocessingImages1(path): #nischay
  """
  input : Path
  output : Pre processed images
  """
  image_data = ImageDataGenerator(zoom_range=0.2, shear_range=0.2,preprocessing_function= preprocess_input , horizontal_flip=True)
  image = image_data.flow_from_directory(directory=path , target_size =(224,224), batch_size= 32, class_mode= 'binary')

  return image

In [ ]:
path="/content/train"
train_data= preprocessingImages1(path)

# for testing data preprOcessed data

In [15]:
def preprocessingImages2(path):
  """
  input : Path
  output : Pre processed images
  """
  image_data = ImageDataGenerator(preprocessing_function= preprocess_input)
  image = image_data.flow_from_directory(directory=path , target_size =(224,224), batch_size= 32, class_mode= 'binary')

  return image

In [16]:
path="/content/test"
test_data=preprocessingImages2(path)

Found 679 images belonging to 2 classes.


In [ ]:
path="/content/val"
val_data=preprocessingImages2(path)

In [18]:
# Early stopping and model check point

from keras.callbacks import ModelCheckpoint , EarlyStopping

#early stopping

es = EarlyStopping(monitor="val_accuracy",min_delta=0.01, patience= 3,verbose=1,mode='auto')

#model check point 
mc = ModelCheckpoint(monitor="val_accuracy",filepath="./bestmodel.h5", save_best_only = True,verbose=1,mode='auto')

cd=[es,mc]

# MODEL TRAINING

In [19]:
import numpy as np
import matplotlib.pyplot as plt
from keras.layers import Flatten, Dense
from keras.models import Model, load_model
from keras.applications.mobilenet import MobileNet
import keras

In [ ]:
base_model = MobileNet(input_shape=(224,224,3), include_top= False)

In [21]:
for layer in base_model.layers:
  layer.trainable = False

In [22]:
X = Flatten()(base_model.output)
X = Dense(units=1, activation='sigmoid')(X)

model = Model(base_model.input, X)

In [ ]:
model.summary()

In [24]:
model.compile(optimizer='rmsprop', loss=keras.losses.binary_crossentropy, metrics=['accuracy'])

In [25]:
## Call back
from keras.callbacks import ModelCheckpoint, EarlyStopping

#model checkpoint
mc = ModelCheckpoint(filepath="bestmodel.h5", monitor='val_accuracy', verbose=1, save_best_only= True)

#Early stopping
es = EarlyStopping(monitor="val_accuracy", min_delta=0.01, patience=3, verbose=1)

cb=[mc, es]

In [ ]:
hist = model.fit_generator(train_data, steps_per_epoch=8, epochs=30, validation_data=val_data, validation_steps=16, callbacks=cb)

In [27]:
#load best fit model
model= load_model("/content/bestmodel.h5")

In [ ]:
acc= model.evaluate_generator(test_data)[1]

print(f"Our model accuracy is: {acc*100} % ")

In [ ]:
h = hist.history
h.keys()

In [ ]:
plt.plot(h['accuracy'])
plt.plot(h['val_accuracy'], c='red')
plt.title("acc vs val-acc")
plt.show

In [ ]:
plt.plot(h['loss'])
plt.plot(h['val_loss'], c='red')
plt.title("loss vs val-loss")
plt.show

In [32]:
from keras.preprocessing import image

In [ ]:
#path for the image to see if it predicts the correct class or not

path = "/content/Brain Tumor Data Set/Brain Tumor Data Set/Brain Tumor/Cancer (1902).jpg"
img = image.load_img(path, target_size=(224, 224) )

i=image.img_to_array(img)/255
#i=preprocess_input(i)
input_arr=np.array([i])
input_arr.shape


predictions=model.predict(input_arr)

#predictions=np.argmax(predictions,axis=1)
print(predictions)

#precentage of cancer 
if predictions > 0.6:
  print("non cancerous")
else :
  print("cancerous")

#to display the image
plt.imshow(input_arr[0])
plt.show()
input_arr.shape
input_arr =np.expand_dims(input_arr, axis=0)